In [1]:
%load_ext autoreload
%autoreload 2

# Standard Library Imports
from datetime import datetime, timedelta

# Third-party imports
import pandas as pd

# Local module imports
from src.inference import load_batch_of_features_from_store, load_model_from_registry, get_model_predictions
from src.feature_store_api import get_feature_store
import src.config as config


In [2]:
current_date = pd.to_datetime(datetime.utcnow()).floor('H')

features = load_batch_of_features_from_store(current_date)

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/111769
Connected. Call `.close()` to terminate connection gracefully.
FROM-TO DATA:
 2023-09-06 13:00:00 2023-10-04 12:00:00 



Fetching data from 2023-09-06 13:00:00 to 2023-10-04 12:00:00


2023-10-04 10:40:37,970 INFO: USE `taxi_demand_api_featurestore`
2023-10-04 10:40:38,903 INFO: SELECT `fg0`.`pickup_hour` `pickup_hour`, `fg0`.`rides` `rides`, `fg0`.`pickup_location_id` `pickup_location_id`
FROM `taxi_demand_api_featurestore`.`time_series_hourly_feature_group_1` `fg0`
WHERE `fg0`.`pickup_hour` >= TIMESTAMP '2023-09-05 01:00:00.000' AND `fg0`.`pickup_hour` < TIMESTAMP '2023-10-05 12:00:00.000'


178080 178080
          pickup_hour  rides  pickup_location_id
0 2023-09-24 19:00:00     14                  41
2 2023-09-18 23:00:00     44                 233
3 2023-09-24 22:00:00      0                 171
4 2023-09-18 10:00:00     20                 158
5 2023-09-11 13:00:00    167                 239
2023-09-06 13:00:00 2023-10-04 12:00:00


In [3]:
model = load_model_from_registry()
predictions = get_model_predictions(model, features)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/111769
Connected. Call `.close()` to terminate connection gracefully.

See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)


In [4]:
predictions['pickup_hour'] = current_date
predictions

,pickup_location_id,predicted_demand,pickup_hour
0,1,2.0,2023-10-04 13:00:00
1,2,0.0,2023-10-04 13:00:00
2,3,0.0,2023-10-04 13:00:00
3,4,4.0,2023-10-04 13:00:00
4,5,0.0,2023-10-04 13:00:00
...,...,...,...
260,261,40.0,2023-10-04 13:00:00
261,262,96.0,2023-10-04 13:00:00
262,263,116.0,2023-10-04 13:00:00
263,264,90.0,2023-10-04 13:00:00


In [5]:
feature_group = get_feature_store().get_or_create_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTIONS,
    version=1,
    description="Predictions generated by the production model",
    primary_key=["pickup_location_id", "pickup_hour"],
    event_time="pickup_hour"
    )

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/111769
Connected. Call `.close()` to terminate connection gracefully.


In [6]:
feature_group.insert(predictions, write_options={"wait_for_job": False})


Uploading Dataframe: 0.00% |          | Rows 0/265 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: model_predictions_feature_group_1_offline_fg_backfill
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/111769/jobs/named/model_predictions_feature_group_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x14be2d870>, None)

In [7]:
predictions

,pickup_location_id,predicted_demand,pickup_hour
0,1,2.0,2023-10-04 13:00:00
1,2,0.0,2023-10-04 13:00:00
2,3,0.0,2023-10-04 13:00:00
3,4,4.0,2023-10-04 13:00:00
4,5,0.0,2023-10-04 13:00:00
...,...,...,...
260,261,40.0,2023-10-04 13:00:00
261,262,96.0,2023-10-04 13:00:00
262,263,116.0,2023-10-04 13:00:00
263,264,90.0,2023-10-04 13:00:00
